# Behavioral Cloning P3
### Self driving cars nanodegree at Udacity

In [1]:
#Imports
#Loading our data
import csv as csv
import cv2
import numpy as np

#Keras imports
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Conv2D, MaxPooling2D, Activation, Cropping2D
from keras import backend as K
from keras.callbacks import EarlyStopping

#Matplotlib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import sklearn

import random

Using TensorFlow backend.


### Loading the data

In order to load our data we firstly read the driving_log.csv file which contains the paths to the left, center and right cameras and also info about steering angle, throttle, break and speed. 

We will want to automate the loading process in order to easily load our own data which is stored in different directories. For that we used several libraries such as csv (read csv files) and cv2 (read images).

In [2]:
#Utils

#Reading the csv file
def readCSV(in_csv_path, in_path):
    lines = []
    with open(in_csv_path) as csvfile:
        reader = csv.reader(csvfile)
        next(reader, None) #Skip header
        for line in reader:
            #Correct images paths
            line_aux = []
            #Center image
            line_aux.append(in_path + line[0].split('/')[-1])
            #Left image
            line_aux.append(in_path + line[1].split('/')[-1])
            #Right image
            line_aux.append(in_path + line[2].split('/')[-1])
            #Steering angle
            line_aux.append(float(line[3]))
            
            lines.append(line_aux)
            
    return lines

#Read center image and steering angle
def readData_Basic(in_lines):
    images = []
    measurements = []
    
    for line in in_lines:
        image = cv2.imread(line[0])
        images.append(image)
        measurements.append(line[3])
    
    return np.array(images), np.array(measurements)

In [3]:
imgs_path = './data/IMG/'
csv_path = './data/driving_log.csv'

csv_lines = readCSV(csv_path, imgs_path)

In [4]:
#Explore the data
csv_line_number = 1

current_line = csv_lines[csv_line_number]
print("Center image path- {}. Left image path- {}. Right image path- {}. Steering Angle- {}.".format(current_line[0], current_line[1], current_line[2], current_line[3]))

Center image path- ./data/IMG/center_2016_12_01_13_30_48_404.jpg. Left image path- ./data/IMG/left_2016_12_01_13_30_48_404.jpg. Right image path- ./data/IMG/right_2016_12_01_13_30_48_404.jpg. Steering Angle- 0.0.


### Data preprocessing

To grayscale?

In [5]:
def image_to_yuv(in_image):
    return cv2.cvtColor(in_image, cv2.COLOR_BGR2YUV)

### Basic neural network using Keras

We will implement a basic neural network in order to verify that everything is working before implementing a more complex model. This network will just going to be a flattened image connected to a single output node. This single output node will predict the stearing angle, thus converting this model into a regression network. In contrast with a classification network, we may apply a softmax activation function to the output layer. Nevertheless in this case we will not use an activation function. We will directly predict the steering measurement. 

For this basic implementation we will use Keras as a library which works with tensorflow as backend. This will simplify our implementation and will be great for prototyping. Let's go ahead!

Improvement 1: In order to improve our model we need to preprocess our input data. For that we will add two preprocessing steps: normalization and  mean centering the data. We will add a lambda layer to our model. After doing this, we can decrease the training epochs a lot. We will fix the number of epochs in 2.

In [6]:
def basic_model():
    #Model definition
    model = Sequential()
    #Lambda layer for normalizing our data. In order to mean center the data, we will
    #need to substract -0.5 (shifting the model down) to the normalized data. 
    model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
    model.add(Flatten(input_shape=(160,320,3)))
    model.add(Dense(1))
    
    return model

### Trying a more complex network such as LeNet-5 architecture

We will implement LeNet-5 architecture using Keras.

In [7]:
def lenet_5():
    model = Sequential()
    #Lambda layer for normalizing our data. In order to mean center the data, we will
    #need to substract -0.5 (shifting the model down) to the normalized data. 
    model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))

    #First set of CONV => RELU => POOL
    model.add(Conv2D(3, (5, 5), input_shape=(160,320,3), activation= 'relu'))
    model.add(MaxPooling2D(pool_size= (2, 2), strides= (2, 2)))

    #Second set of CONV => RELU => POOL
    model.add(Conv2D(6, (5, 5), activation= 'relu'))
    model.add(MaxPooling2D(pool_size= (2, 2), strides= (2, 2)))

    #Setting the FCs layers
    model.add(Flatten())
    model.add(Dense(120))
    model.add(Dense(84))

    #Output layer
    model.add(Dense(1))
    
    return model

In [8]:
#Loading data with center image and measurements
X_train_basic, y_train_basic = readData_Basic(csv_lines)
print('Data loaded successfully!')

print("Training set shape: ", X_train_basic.shape)
print("Training set labels shape: ", y_train_basic.shape)


#RUNNING BASIC MODEL
print("BASIC MODEL training")

basic_model = basic_model()
#Model compilation
#For the loss function we will use Mean Squared Error (MSE). We will minimize the 
#error between the steering measurement which the network predicts and the ground 
#truth steering measurements provided by the dataset
basic_model.compile(loss='mse', optimizer='adam')
#we also shuffle the data and split off 20% of the data to use for a validation set
basic_model.fit(X_train_basic, y_train_basic, validation_split=0.2, shuffle=True, epochs= 2)

#Keras by default will run 10 epochs. Nevertheless with 10 epochs we will 
#overfit the training data. For that reason we will only perform 6 epochs
basic_model.save('basic_model.h5')

K.clear_session()

print("******************************************")
print()

#RUNNING LENET-5 MODEL
print("LENET-5 MODEL training")

lenet5_model = lenet_5()

#Model compilation
lenet5_model.compile(loss='mse', optimizer='adam')
lenet5_model.fit(X_train_basic, y_train_basic, validation_split=0.2, shuffle=True, epochs= 2)
lenet5_model.save('lenet_model.h5')

K.clear_session()

print("******************************************")
print()



Data loaded successfully!
Training set shape:  (8036, 160, 320, 3)
Training set labels shape:  (8036,)
BASIC MODEL training
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 6428 samples, validate on 1608 samples
Epoch 1/2
6428/6428 [==============================] - 3s 530us/step - loss: 2.0965 - val_loss: 1.3814
Epoch 2/2
6428/6428 [==============================] - 2s 376us/step - loss: 2.9891 - val_loss: 3.1842
******************************************

LENET-5 MODEL training
Train on 6428 samples, validate on 1608 samples
Epoch 1/2
6428/6428 [==============================] - 6s 865us/step - loss: 1.0779 - val_loss: 0.0139
Epoch 2/2
6428/6428 [==============================] - 4s 634us/step - loss: 0.0113 - val_loss: 0.0118
******************************************



### Data augmentation

In order to increase the number of data we have driven the car in the opposite direction along the routes. At the same time we can flip the images and also taking the opposite sign of steering measurements

In [9]:
def flipImg_invertMeas(in_image, in_measurement):
    image_flipped = np.fliplr(in_image)
    measurement_inverted = -in_measurement
    
    return image_flipped, measurement_inverted

def augmentDataset(in_images, in_measurements, how_much= 0.3):
    augmented_num = int(len(in_images) * how_much)
    
    #print("We will generate {} images and measurements.".format(augmented_num))
    
    sklearn.utils.shuffle(in_images, in_measurements)

    in_images = in_images[:augmented_num]
    in_measurements = in_measurements[:augmented_num]
    
    if (len(in_images) == len(in_measurements)):
        for index in range(0, len(in_images)):
            current_img = in_images[index]
            current_meas = in_measurements[index]
            in_images[index], in_measurements[index] = flipImg_invertMeas(current_img, current_meas)
    else:
        print("Shouldn't be here!")
        return 0
    
    return in_images, in_measurements

### Using Multiple Cameras

Up to this point we only used the center camera. But, using side cameras should be a great decision because we will have three times more data. And also, using these images we will teach the network how to steer back to the center if the vehicle starts drifting off to the side. 

The simulator captures images from three cameras mounted on the car: a center, right and left camera. That’s because of the issue of recovering from being off-center. In the simulator, you can weave all over the road and turn recording on and off to record recovery driving. In a real car, however, that’s not really possible. At least not legally.


In [10]:
def readData_Advanced(in_line, correction=0.25):
    images = []
    measurements = []
    
    center_img = cv2.imread(in_line[0])
    left_img = cv2.imread(in_line[1])
    right_img = cv2.imread(in_line[2])

    #center_img = preprocess_image(center_img)
    #left_img = preprocess_image(left_img)
    #right_img = preprocess_image(right_img)

    images.append(center_img)
    images.append(left_img)
    images.append(right_img)

    #Measurements
    center_steer = in_line[3]
    left_steer = center_steer + correction
    right_steer = center_steer - correction

    measurements.append(center_steer)
    measurements.append(left_steer)
    measurements.append(right_steer)
    
    return images, measurements

### nVidia Model



In [11]:
def NvidiaArchitecture():
    model = Sequential()
    #Preprocessing
    model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160, 320, 3)))
    model.add(Cropping2D(cropping=((50,20), (0,0))))
    
    model.add(Conv2D(24,(5,5), strides=(2,2), activation='relu'))
    model.add(Conv2D(36,(5,5), strides=(2,2), activation='relu'))
    model.add(Conv2D(48,(5,5), strides=(2,2), activation='relu'))
    model.add(Conv2D(64,(3,3), activation='relu'))
    model.add(Conv2D(64,(3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))
    
    return model
    

In [12]:
def generator(samples, batch_size=32, data_augmentation=False):
    num_samples = len(samples)
    
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        
        images_batch = []
        measurements_batch = []
        
        for offset in range(0, num_samples, batch_size):
            
            batch_samples = samples[offset:offset+batch_size]
      
            for batch_sample in batch_samples:
                images_sample, measurements_sample = readData_Advanced(batch_sample)
                images_batch.extend(images_sample)
                measurements_batch.extend(measurements_sample)
            
            if data_augmentation:
                images_augm, measurements_augm = augmentDataset(images_batch, measurements_batch, how_much= 0.3)
                images_batch.extend(images_augm)
                measurements_batch.extend(measurements_augm)
        
        
        yield sklearn.utils.shuffle(np.asarray(images_batch), np.asarray(measurements_batch))

In [ ]:
train_samples, validation_samples = train_test_split(csv_lines, test_size= 0.2)

print("We will train with {} samples.".format(len(train_samples)))
print("We will validate with {} samples.".format(len(validation_samples)))

train_generator = generator(train_samples, batch_size= 32, data_augmentation= False)
validation_generator = generator(validation_samples, batch_size= 32, data_augmentation= False)

model = NvidiaArchitecture()

model.compile(loss='mse', optimizer='adam')
model.summary() 

early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=0, mode='auto')
model.fit_generator(train_generator, samples_per_epoch=24000, nb_epoch=28, validation_data=validation_generator, nb_val_samples=1024)#, callbacks=[early_stop])

model.save('nvidia_model.h5')

We will train with 6428 samples.
We will validate with 1608 samples.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_3 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_3 (Cropping2D)    (None, 90, 320, 3)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 43, 158, 24)       1824      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 20, 77, 36)        21636     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 8, 37, 48)         43248     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 6, 35, 64)         27712     
_________________________________________________________________
conv2d_

/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:15: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  from ipykernel import kernelapp as app
/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=1024, epochs=28, steps_per_epoch=24000, validation_data=<generator...)`
  from ipykernel import kernelapp as app


Epoch 1/28
